<a href="https://colab.research.google.com/github/Muzsek/One_Number/blob/main/O_N_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install captcha

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 2.8 MB/s eta 0:00:00


In [ ]:
import torch
from captcha.image import ImageCaptcha
import matplotlib.pyplot as plt
import string
import random
from pathlib import Path
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from torch import nn

In [ ]:
!mkdir data
!mkdir data/train
!mkdir data/test

In [ ]:
train_path = Path('./data/train')
# print(train_path)
test_path = Path('./data/test')
# print(test_path)

chars = string.digits

image = ImageCaptcha(width = 120, height = 50)

for i in range(int(1), int(5001)):
  label = ''.join(random.choices(chars))
  img = image.generate_image(label)
  img.save(train_path / f'{label}_{i}.png')
for i in range(int(1), int(1001)):
  label = ''.join(random.choices(chars))
  img = image.generate_image(label)
  img.save(test_path / f'{label}_{i}.png')

In [ ]:
!rm -rf ./data/train/*
!rm -rf ./data/test/*

In [ ]:
print(f"First train image: {os.listdir(train_path)[0]}\n",
      f"Train images count: {len(os.listdir(train_path))}")
print(f"First test image: {os.listdir(test_path)[0]}\n",
      f"Test images count: {len(os.listdir(test_path))}")

First train image: 1_2791.png
 Train images count: 5000
First test image: 6_139.png
 Test images count: 1000


In [ ]:
training_data = list(train_path.glob('*'))
testing_data = list(test_path.glob('*'))

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
class CustomDataset(Dataset):
  def __init__(self,image_paths,transform):
    self.image_paths = image_paths
    self.transform = transform

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, index):
    image_paths = self.image_paths[index]
    image = Image.open(image_paths).convert('RGB')
    label_str = image_paths.stem.split("_")[0]
    label = int(label_str)
    return self.transform(image),label

In [ ]:
training_dataset = CustomDataset(training_data, transform = transform)
testing_dataset = CustomDataset(testing_data, transform = transform)

In [ ]:
print(f"{training_dataset}")
print(f"{testing_dataset}")

In [ ]:
testing_dataset[0][1]

6

In [ ]:
training_loader = DataLoader(training_dataset, batch_size = 64, shuffle = True)
testing_loader = DataLoader(testing_dataset, batch_size = 64, shuffle = False)

In [ ]:
for images, labels in training_loader:
    print("Images batch shape:", images.shape)
    print("Labels batch:", labels)
    break

Images batch shape: torch.Size([64, 3, 50, 120])
Labels batch: tensor([4, 4, 3, 8, 2, 3, 0, 2, 3, 5, 9, 5, 9, 0, 4, 1, 9, 1, 8, 6, 5, 4, 7, 1,
        2, 2, 9, 8, 7, 5, 6, 5, 8, 2, 2, 3, 8, 9, 4, 0, 8, 5, 2, 1, 7, 8, 8, 8,
        8, 1, 1, 8, 5, 5, 6, 3, 0, 7, 1, 0, 1, 1, 4, 8])


In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1_block = nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 16, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.Conv2d(in_channels = 16, out_channels = 16, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        nn.BatchNorm2d(16)
    )
    self.conv2_block = nn.Sequential(
        nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        nn.BatchNorm2d(32)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = 32 * 12 * 30, out_features = 10)
    )

  def forward(self, x):
    # x = self.conv1_block(x)
    # print(x.shape)
    # x = self.conv2_block(x)
    # print(x.shape)
    # x = self.classifier(x)
    # print(f"After flatten: {x.shape}")
    # return x
    return self.classifier(self.conv2_block(self.conv1_block(x)))

device = "cuda" if torch.cuda.is_available() else "cpu"
model_0 = CNN().to(device)
next(model_0.parameters()).is_cuda

False

In [ ]:
epochs = 3
loss_fn = nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(params = model_0.parameters(), lr = 1E-3)

In [ ]:
for epoch in range(epochs):
    print(f"Epoch: {epoch}\n----------")
    model_0.train()
    train_loss = 0

    for batch, (X, y) in enumerate(training_loader):
        X, y = X.to(device), y.to(device)
        y_pred = model_0(X)
        loss = loss_fn(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch % 50 == 0:
            print(f"Batch: {batch}, Loss: {loss.item():.4f}")

    train_loss /= len(training_loader)

    # Tesztelés az epoch végén
    test_loss, test_acc = 0, 0
    model_0.eval()
    with torch.inference_mode():
        for X, y in testing_loader:
            X, y = X.to(device), y.to(device)
            test_pred = model_0(X)
            test_loss += loss_fn(test_pred, y).item()
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))

    test_loss /= len(testing_loader)
    test_acc /= len(testing_loader)

    print(f"\nTrain loss: {train_loss:.5f} | Test loss: {test_loss:.5f} | Test acc: {test_acc:.2f}%\n")


Epoch: 0
----------
Batch: 0, Loss: 0.5046
Batch: 50, Loss: 0.0830

Train loss: 0.15460 | Test loss: 0.08198 | Test acc: 97.99%

Epoch: 1
----------
Batch: 0, Loss: 0.0242
Batch: 50, Loss: 0.0075

Train loss: 0.01701 | Test loss: 0.05537 | Test acc: 98.83%

Epoch: 2
----------
Batch: 0, Loss: 0.0066
Batch: 50, Loss: 0.0043

Train loss: 0.00542 | Test loss: 0.03762 | Test acc: 99.51%

